# 0. 개요

- lag1부터 시작하는, lag2 부터 시작하는.... 28개의 모델을 만드는 모델을 만들어보자.

- 모든 시간대를 대상으로 하는 것이 아닌, 예측해야하는 시간대의 target이 비슷한 데이터를 바탕으로
- 2016-04-25 ~ 2016-05-22 : d_1914 ~ d_1941 (public)
- 2016-05-23 ~ 2016-06-19 : d_1942 ~ d_1969 (private)
- 모델을 제작

# 1. 라이브러리 로드 및 데이터 불러오기

In [1]:
import pandas as pd
import numpy as np 
from sklearn.ensemble import RandomForestRegressor
from lightgbm import LGBMRegressor
from sklearn import metrics
from sklearn.model_selection import TimeSeriesSplit, KFold

import matplotlib.pyplot as plt
import seaborn as sns
import pickle 
import time
import datetime
import os

from scipy.sparse import csr_matrix

from mypackage import *

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 400)

### <div style="color:red">파일명 체크!!!</div>
- all_df: 기초작업 수준. 
- all_df4: lag 데이터 엄청 많이 만들어 놓은 것.
- all_df5: 'id', 'item_id', 'dept_id', 'cat_id', 'store_id', 'state_id', 'd', 'sales', 'date', 'wm_yr_wk', 'weekday', 'month', 'year', 'event_name_1', 'event_type_1', 'snap_CA', 'snap_TX', 'snap_WI', 'is_event', 'day', 'week', 'sell_price', 'revenue', sales_rolling_mean_t7', 'rolling_mean_t7', 'rolling_std_t7', 'sales_rolling_mean_t28', 'rolling_mean_t28', 'rolling_std_t28', 'sales_rolling_mean_t56', 'rolling_mean_t56', 'rolling_std_t56', 'sales_rolling_mean_t112','rolling_mean_t112', 'rolling_std_t112', 'sales_rolling_mean_t168', 'rolling_mean_t168', 'rolling_std_t168', 'lag_t28', 'lag_t29', 'lag_t30', 'lag_t31', 'lag_t32', 'lag_t33', 'lag_t34', 'revenue_lag_t28', 'revenue_lag_t29', 'revenue_lag_t30','revenue_lag_t31', 'revenue_lag_t32', 'revenue_lag_t33', 'revenue_lag_t34'

In [2]:
print('data loading')
with open('inputs/all_df4.pickle', 'rb') as f:
    all_df = pickle.load(f)
print('data loaded')

data loading
data loaded


In [31]:
all_df.columns = ['id', 'item_id', 'dept_id', 'cat_id', 'store_id', 'state_id', 'd',
       'sales', 'date', 'wm_yr_wk', 'weekday', 'month', 'year',
       'event_name_1', 'event_type_1', 'snap_CA', 'snap_TX', 'snap_WI',
       'is_event', 'day', 'week', 'sell_price', 'lag_t1', 'lag_t2', 'lag_t3',
       'lag_t4', 'lag_t5', 'lag_t6', 'lag_t7', 'lag_t8', 'lag_t9', 'lag_t10',
       'lag_t11', 'lag_t12', 'lag_t13', 'lag_t14', 'lag_t15', 'lag_t16',
       'lag_t17', 'lag_t18', 'lag_t19', 'lag_t20', 'lag_t21', 'lag_t22',
       'lag_t23', 'lag_t24', 'lag_t25', 'lag_t26', 'lag_t27', 'lag_t28',
       'rolling_mean_t7', 'rolling_std_t7', 'rolling_mean_t28',
       'rolling_std_t28', 'rolling_mean_t56', 'rolling_std_t56',
       'rolling_mean_t84', 'rolling_std_t84', 'rolling_mean_t112',
       'rolling_std_t112', 'rolling_mean_t168', 'rolling_std_t168',
       'price_change_t1', 'price_change_t365', 'rolling_price_std_t7',
       'rolling_price_std_t28']

In [32]:
for i in range(29, 33):
    all_df[f'lag_t{i}'] = all_df.groupby(['id'])['sales'].transform(lambda x: x.shift(i))



In [5]:
all_df['rolling_price_max_t180'] = all_df.groupby(['id'])['sell_price'].transform(lambda x: x.shift(1).rolling(365).max())

all_df['price_change_t180'] = (all_df['rolling_price_max_t180'] - all_df['sell_price']) / (all_df['rolling_price_max_t180'])

del all_df['rolling_price_max_t180']

In [ ]:
all_df['rolling_price_max_t365'] = all_df.groupby(['id'])['sell_price'].transform(lambda x: x.shift(1).rolling(365).max())

all_df['price_diff_t365'] = all_df['rolling_price_max_t365'] - all_df['sell_price']

del all_df['rolling_price_max_t365']

In [ ]:
all_df['rolling_price_max_t180'] = all_df.groupby(['id'])['sell_price'].transform(lambda x: x.shift(1).rolling(365).max())

all_df['price_change_t180']a = (all_df['rolling_price_max_t180'] - all_df['sell_price']) / (all_df['rolling_price_max_t180'])

del all_df['rolling_price_max_t180']

In [ ]:
all_df['rolling_price_max_t60'] = all_df.groupby(['id'])['sell_price'].transform(lambda x: x.shift(1).rolling(365).max())

all_df['price_change_t60'] = (all_df['rolling_price_max_t60'] - all_df['sell_price']) / (all_df['rolling_price_max_t60'])

del all_df['rolling_price_max_t60']

In [ ]:
all_df['rolling_price_max_t28'] = all_df.groupby(['id'])['sell_price'].transform(lambda x: x.shift(1).rolling(365).max())

all_df['price_change_t28'] = (all_df['rolling_price_max_t28'] - all_df['sell_price']) / (all_df['rolling_price_max_t28'])

del all_df['rolling_price_max_t28']

In [ ]:
all_df['rolling_price_std_t56'] = all_df.groupby(['id'])['sell_price'].transform(lambda x: x.rolling(56).std())
all_df['rolling_price_std_t112'] = all_df.groupby(['id'])['sell_price'].transform(lambda x: x.rolling(112).std())
all_df['rolling_price_std_t168'] = all_df.groupby(['id'])['sell_price'].transform(lambda x: x.rolling(168).std())


# 2. 데이터 탐색

여기서는 test셋과 비슷한 시간대를 찾아서, 이 데이터만을 바탕으로 모델을 만들것임.  

test: 2016-04-25 ~ 2016-05-22  17~20주차 (16주차나 21주차까지 늘려야될지도 생각해보자)

---
train: 2015-04-27 ~ 2015-05-24  
train: 2014-04-28 ~ 2014-05-25  / 2014-04-21 ~ 2014-05-18
train: 2013-04-27 ~ 2013-05-24  
train: 2012-04-27 ~ 2012-05-24 
train: 2011-04-27 ~ 2011-05-24 

In [ ]:
len(all_df)

In [ ]:
all_df.tail()

In [33]:
# c00 = all_df['week'] == 9
# c01 = all_df['week'] == 10
# c02 = all_df['week'] == 11
# c03 = all_df['week'] == 12

c1 = all_df['week'] == 13
c2 = all_df['week'] == 14
c3 = all_df['week'] == 15
c4 = all_df['week'] == 16

c5 = all_df['week'] == 17
c6 = all_df['week'] == 18
c7 = all_df['week'] == 19
c8 = all_df['week'] == 20
all_df17_20 = all_df[c1 | c2 | c3 | c4 | c5 | c6 | c7 | c8]

In [5]:
all_df17_20.tail()

,id,item_id,dept_id,cat_id,store_id,state_id,d,target,date,wm_yr_wk,weekday,month,year,event_name_1,event_type_1,snap_CA,snap_TX,snap_WI,is_event,day,week,sell_price,lag_t1,lag_t2,lag_t3,lag_t4,lag_t5,lag_t6,lag_t7,lag_t8,lag_t9,lag_t10,lag_t11,lag_t12,lag_t13,lag_t14,lag_t15,lag_t16,lag_t17,lag_t18,lag_t19,lag_t20,lag_t21,lag_t22,lag_t23,lag_t24,lag_t25,lag_t26,lag_t27,lag_t28,rolling_mean_t7,rolling_std_t7,rolling_mean_t28,rolling_std_t28,rolling_mean_t56,rolling_std_t56,rolling_mean_t84,rolling_std_t84,rolling_mean_t112,rolling_std_t112,rolling_mean_t168,rolling_std_t168,price_change_t1,price_change_t365,rolling_price_std_t7,rolling_price_std_t28
853715,FOODS_3_823_WI_3_validation,1432,2,0,9,2,1047,0,2016-05-22,11617,3,5,2016,30,4,0,0,0,0,22,20,2.980469,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.285645,0.488037,0.214233,0.567871,0.232178,0.571777,0.404785,0.823242,0.508789,1.013672,0.613281,0.990723,0.0,0.0,0.000000e+00,0.000000e+00
853716,FOODS_3_824_WI_3_validation,1433,2,0,9,2,1047,0,2016-05-22,11617,3,5,2016,30,4,0,0,0,0,22,20,2.480469,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.142822,0.377930,0.321533,0.547852,0.160767,0.416748,0.107117,0.347656,0.080383,0.304199,0.053558,0.250977,0.0,0.0,5.960464e-08,0.000000e+00
853717,FOODS_3_825_WI_3_validation,1434,2,0,9,2,1047,0,2016-05-22,11617,3,5,2016,30,4,0,0,0,0,22,20,3.980469,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.571289,0.786621,0.893066,1.166016,1.071289,1.059570,0.928711,0.979004,0.893066,0.998535,0.797852,0.951172,0.0,0.0,0.000000e+00,5.960464e-08
853718,FOODS_3_826_WI_3_validation,1435,2,0,9,2,1047,0,2016-05-22,11617,3,5,2016,30,4,0,0,0,0,22,20,1.280273,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,1.142578,1.344727,0.928711,1.051758,1.088867,1.066406,1.023438,1.097656,1.161133,1.326172,1.315430,1.567383,0.0,0.0,0.000000e+00,0.000000e+00
853719,FOODS_3_827_WI_3_validation,1436,2,0,9,2,1047,0,2016-05-22,11617,3,5,2016,30,4,0,0,0,0,22,20,1.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,1.071289,1.719727,1.517578,1.778320,1.536133,1.839844,1.588867,1.876953,1.494141,1.771484,0.0,0.0,0.000000e+00,0.000000e+00


In [35]:
from sklearn.preprocessing import LabelEncoder

le_id = LabelEncoder()
all_df17_20['id'] = le_id.fit_transform(all_df17_20['id'])

c:\users\kjb\data_analysis\10_estimate_sales_of_walmart_retail_goods\venv\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [37]:
all_df2 = all_df17_20.drop(['d', 'sales', 'date', 'wm_yr_wk'], axis = 1)

In [43]:
all_df2_X = all_df17_20[all_df17_20['date'] <= '2016-04-24']

In [77]:
test_date = ['2016-04-25', '2016-04-26', '2016-04-27', '2016-04-28',
       '2016-04-29', '2016-04-30', '2016-05-01', '2016-05-02',
       '2016-05-03', '2016-05-04', '2016-05-05', '2016-05-06',
       '2016-05-07', '2016-05-08', '2016-05-09', '2016-05-10',
       '2016-05-11', '2016-05-12', '2016-05-13', '2016-05-14',
       '2016-05-15', '2016-05-16', '2016-05-17', '2016-05-18',
       '2016-05-19', '2016-05-20', '2016-05-21', '2016-05-22']

cols = ['id', 'item_id', 'dept_id', 'cat_id', 'store_id', 'state_id', 'weekday',
        'month', 'year', 'event_name_1', 'event_type_1', 'snap_CA', 'snap_TX',
        'snap_WI', 'day', 'week', 'sell_price',
        'rolling_mean_t7', 'rolling_std_t7', 'rolling_mean_t28',
        'rolling_std_t28', 'rolling_mean_t56', 'rolling_std_t56',
        'rolling_mean_t84', 'rolling_mean_t168', 
        'price_change_t1', 'price_change_t365', 'rolling_price_std_t7',
        'rolling_price_std_t28']

train_y = all_df2_X.sales
results = []

for i in range(len(test_date)):
    print(i+1)
    cols2 = cols.copy()
    for j in range(i+1, i+5):
        cols2.append(f'lag_t{j}')
    train_X = all_df2_X[cols2]
    test = all_df17_20[all_df17_20['date'] == test_date[i]]
    test = test[cols2]
    
    n_fold = 5
    folds = KFold(n_splits=n_fold, shuffle=True)
    splits = folds.split(train_X, train_y)

    y_preds = np.zeros(test.shape[0])
    y_oof = np.zeros(train_X.shape[0])

    feature_importances = pd.DataFrame()
    feature_importances['feature'] = train_X.columns
    mean_score = []

    print(train_X.columns)

    for fold_n, (train_index, valid_index) in enumerate(splits):
        print('Fold:',fold_n+1)

        X_train, X_valid = train_X.iloc[train_index], train_X.iloc[valid_index]
        y_train, y_valid = train_y.iloc[train_index], train_y.iloc[valid_index]

        lgb = LGBMRegressor(
            objective = 'regression',
            boosting_type = 'gbdt',
            num_leaves = 4096,
            colsample_bytree = 0.8,
            subsample = 0.8,
            n_estimators = 10, ## 중요!!!!
            learning_rate = 0.1,
            n_jobs = -1,
            device = 'gpu',
            reg_lambda = 0.1
        )
        lgb.fit(X_train, y_train, eval_set=[(X_valid, y_valid)], early_stopping_rounds = 20, verbose = True)

        feature_importances[f'fold_{fold_n + 1}'] = lgb.feature_importances_

#         y_pred_valid = lgb.predict(X_valid, num_iteration=lgb.best_iteration_)
#         y_oof[valid_index] = y_pred_valid
        # validation 측정
#         val_score = np.sqrt(metrics.mean_squared_error(y_pred_valid, y_valid))
#         print(f'val rmse score is {val_score}')
#         mean_score.append(val_score)
#         all_pred = lgb.predict(var_set_X, num_iteration=lgb.best_iteration_)
#         all_var_score = np.sqrt(metrics.mean_squared_error(all_pred, var_set_y))
#         print(f'2015-04-27 부터 2015-05-22까지 데이터로 validation rmse 결과: {all_var_score}')


        # 예측
        y_preds += lgb.predict(test, num_iteration=lgb.best_iteration_) / n_fold

        # 메모리 정리
        del X_train, X_valid, y_train, y_valid


#     params = lgb.get_params()
#     eval_results = lgb.evals_result_['valid_0']['l2']

#     print('mean rmse score over folds is',np.mean(mean_score))
    results.append(y_preds)


    

test_sales = []
for i in results:
    for j in i:
        test_sales.append(j)

1
Index(['id', 'item_id', 'dept_id', 'cat_id', 'store_id', 'state_id', 'weekday',
       'month', 'year', 'event_name_1', 'event_type_1', 'snap_CA', 'snap_TX',
       'snap_WI', 'is_event', 'day', 'week', 'sell_price', 'rolling_mean_t7',
       'rolling_std_t7', 'rolling_mean_t28', 'rolling_std_t28',
       'rolling_mean_t56', 'rolling_std_t56', 'rolling_mean_t84',
       'rolling_std_t84', 'rolling_mean_t112', 'rolling_std_t112',
       'rolling_mean_t168', 'rolling_std_t168', 'price_change_t1',
       'price_change_t365', 'rolling_price_std_t7', 'rolling_price_std_t28',
       'lag_t1', 'lag_t2', 'lag_t3', 'lag_t4'],
      dtype='object')
Fold: 1
[1]	valid_0's l2: 11.8663
Training until validation scores don't improve for 20 rounds
[2]	valid_0's l2: 10.4391
[3]	valid_0's l2: 9.20356
[4]	valid_0's l2: 8.22339
[5]	valid_0's l2: 7.42522
[6]	valid_0's l2: 6.76458
[7]	valid_0's l2: 6.21828
[8]	valid_0's l2: 5.78438
[9]	valid_0's l2: 5.43837
[10]	valid_0's l2: 5.13985
Did not meet early st

c:\users\kjb\data_analysis\10_estimate_sales_of_walmart_retail_goods\venv\lib\site-packages\numpy\core\fromnumeric.py:3335: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
c:\users\kjb\data_analysis\10_estimate_sales_of_walmart_retail_goods\venv\lib\site-packages\numpy\core\_methods.py:161: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


mean rmse score over folds is nan
2
Index(['id', 'item_id', 'dept_id', 'cat_id', 'store_id', 'state_id', 'weekday',
       'month', 'year', 'event_name_1', 'event_type_1', 'snap_CA', 'snap_TX',
       'snap_WI', 'is_event', 'day', 'week', 'sell_price', 'rolling_mean_t7',
       'rolling_std_t7', 'rolling_mean_t28', 'rolling_std_t28',
       'rolling_mean_t56', 'rolling_std_t56', 'rolling_mean_t84',
       'rolling_std_t84', 'rolling_mean_t112', 'rolling_std_t112',
       'rolling_mean_t168', 'rolling_std_t168', 'price_change_t1',
       'price_change_t365', 'rolling_price_std_t7', 'rolling_price_std_t28',
       'lag_t2', 'lag_t3', 'lag_t4', 'lag_t5'],
      dtype='object')
Fold: 1
[1]	valid_0's l2: 11.7998
Training until validation scores don't improve for 20 rounds
[2]	valid_0's l2: 10.4124
[3]	valid_0's l2: 9.24214
[4]	valid_0's l2: 8.31818
[5]	valid_0's l2: 7.5583
[6]	valid_0's l2: 6.91839
[7]	valid_0's l2: 6.38993
[8]	valid_0's l2: 5.98073
[9]	valid_0's l2: 5.64766
[10]	valid_0's 

Fold: 1
[1]	valid_0's l2: 11.59
Training until validation scores don't improve for 20 rounds
[2]	valid_0's l2: 10.2277
[3]	valid_0's l2: 9.11011
[4]	valid_0's l2: 8.20463
[5]	valid_0's l2: 7.47176
[6]	valid_0's l2: 6.87563
[7]	valid_0's l2: 6.38249
[8]	valid_0's l2: 5.98618
[9]	valid_0's l2: 5.66634
[10]	valid_0's l2: 5.39858
Did not meet early stopping. Best iteration is:
[10]	valid_0's l2: 5.39858
Fold: 2
[1]	valid_0's l2: 11.6008
Training until validation scores don't improve for 20 rounds
[2]	valid_0's l2: 10.2219
[3]	valid_0's l2: 9.08754
[4]	valid_0's l2: 8.1738
[5]	valid_0's l2: 7.43079
[6]	valid_0's l2: 6.82923
[7]	valid_0's l2: 6.32693
[8]	valid_0's l2: 5.92292
[9]	valid_0's l2: 5.59432
[10]	valid_0's l2: 5.32343
Did not meet early stopping. Best iteration is:
[10]	valid_0's l2: 5.32343
Fold: 3
[1]	valid_0's l2: 11.6503
Training until validation scores don't improve for 20 rounds
[2]	valid_0's l2: 10.2757
[3]	valid_0's l2: 9.13457
[4]	valid_0's l2: 8.22265
[5]	valid_0's l2: 7.

[2]	valid_0's l2: 10.2936
[3]	valid_0's l2: 9.18303
[4]	valid_0's l2: 8.2833
[5]	valid_0's l2: 7.55079
[6]	valid_0's l2: 6.95107
[7]	valid_0's l2: 6.458
[8]	valid_0's l2: 6.06403
[9]	valid_0's l2: 5.73694
[10]	valid_0's l2: 5.46303
Did not meet early stopping. Best iteration is:
[10]	valid_0's l2: 5.46303
Fold: 2
[1]	valid_0's l2: 11.8799
Training until validation scores don't improve for 20 rounds
[2]	valid_0's l2: 10.5274
[3]	valid_0's l2: 9.41223
[4]	valid_0's l2: 8.50618
[5]	valid_0's l2: 7.76925
[6]	valid_0's l2: 7.16351
[7]	valid_0's l2: 6.65857
[8]	valid_0's l2: 6.25814
[9]	valid_0's l2: 5.93246
[10]	valid_0's l2: 5.6575
Did not meet early stopping. Best iteration is:
[10]	valid_0's l2: 5.6575
Fold: 3
[1]	valid_0's l2: 11.8454
Training until validation scores don't improve for 20 rounds
[2]	valid_0's l2: 10.4973
[3]	valid_0's l2: 9.37257
[4]	valid_0's l2: 8.46647
[5]	valid_0's l2: 7.72367
[6]	valid_0's l2: 7.11845
[7]	valid_0's l2: 6.61399
[8]	valid_0's l2: 6.21294
[9]	valid_0's

...

Fold: 2
[1]	valid_0's l2: 11.9881
Training until validation scores don't improve for 20 rounds
[2]	valid_0's l2: 10.6292
[3]	valid_0's l2: 9.51831
[4]	valid_0's l2: 8.60941
[5]	valid_0's l2: 7.86066
[6]	valid_0's l2: 7.25357
[7]	valid_0's l2: 6.75278
[8]	valid_0's l2: 6.35083
[9]	valid_0's l2: 6.01035
[10]	valid_0's l2: 5.73876
Did not meet early stopping. Best iteration is:
[10]	valid_0's l2: 5.73876
Fold: 3
[1]	valid_0's l2: 11.7004
Training until validation scores don't improve for 20 rounds
[2]	valid_0's l2: 10.3967
[3]	valid_0's l2: 9.32538
[4]	valid_0's l2: 8.45575
[5]	valid_0's l2: 7.74527
[6]	valid_0's l2: 7.17463
[7]	valid_0's l2: 6.70069
[8]	valid_0's l2: 6.31663
[9]	valid_0's l2: 6.00112
[10]	valid_0's l2: 5.74688
Did not meet early stopping. Best iteration is:
[10]	valid_0's l2: 5.74688
Fold: 4
[1]	valid_0's l2: 12.2748
Training until validation scores don't improve for 20 rounds
[2]	valid_0's l2: 10.9177
[3]	valid_0's l2: 9.79191
[4]	valid_0's l2: 8.87309
[5]	valid_0's l2:

[2]	valid_0's l2: 10.6038
[3]	valid_0's l2: 9.52058
[4]	valid_0's l2: 8.64002
[5]	valid_0's l2: 7.90827
[6]	valid_0's l2: 7.32169
[7]	valid_0's l2: 6.82859
[8]	valid_0's l2: 6.42767
[9]	valid_0's l2: 6.09137
[10]	valid_0's l2: 5.81728
Did not meet early stopping. Best iteration is:
[10]	valid_0's l2: 5.81728
Fold: 3
[1]	valid_0's l2: 11.6448
Training until validation scores don't improve for 20 rounds
[2]	valid_0's l2: 10.3335
[3]	valid_0's l2: 9.24062
[4]	valid_0's l2: 8.35952
[5]	valid_0's l2: 7.62435
[6]	valid_0's l2: 7.04141
[7]	valid_0's l2: 6.55218
[8]	valid_0's l2: 6.14992
[9]	valid_0's l2: 5.8173
[10]	valid_0's l2: 5.55037
Did not meet early stopping. Best iteration is:
[10]	valid_0's l2: 5.55037
Fold: 4
[1]	valid_0's l2: 12.2155
Training until validation scores don't improve for 20 rounds
[2]	valid_0's l2: 10.8524
[3]	valid_0's l2: 9.72212
[4]	valid_0's l2: 8.80542
[5]	valid_0's l2: 8.04946
[6]	valid_0's l2: 7.44955
[7]	valid_0's l2: 6.93958
[8]	valid_0's l2: 6.52419
[9]	valid

[5]	valid_0's l2: 7.8166
[6]	valid_0's l2: 7.2323
[7]	valid_0's l2: 6.74056
[8]	valid_0's l2: 6.33836
[9]	valid_0's l2: 5.99515
[10]	valid_0's l2: 5.7277
Did not meet early stopping. Best iteration is:
[10]	valid_0's l2: 5.7277
Fold: 3
[1]	valid_0's l2: 12.0517
Training until validation scores don't improve for 20 rounds
[2]	valid_0's l2: 10.7425
[3]	valid_0's l2: 9.66264
[4]	valid_0's l2: 8.76943
[5]	valid_0's l2: 8.04503
[6]	valid_0's l2: 7.46808
[7]	valid_0's l2: 6.98732
[8]	valid_0's l2: 6.58638
[9]	valid_0's l2: 6.25312
[10]	valid_0's l2: 5.98722
Did not meet early stopping. Best iteration is:
[10]	valid_0's l2: 5.98722
Fold: 4
[1]	valid_0's l2: 11.9647
Training until validation scores don't improve for 20 rounds
[2]	valid_0's l2: 10.6195
[3]	valid_0's l2: 9.51905
[4]	valid_0's l2: 8.60502
[5]	valid_0's l2: 7.86083
[6]	valid_0's l2: 7.27947
[7]	valid_0's l2: 6.78674
[8]	valid_0's l2: 6.37794
[9]	valid_0's l2: 6.03959
[10]	valid_0's l2: 5.77405
Did not meet early stopping. Best ite

[8]	valid_0's l2: 6.5332
[9]	valid_0's l2: 6.19602
[10]	valid_0's l2: 5.93411
Did not meet early stopping. Best iteration is:
[10]	valid_0's l2: 5.93411
Fold: 3
[1]	valid_0's l2: 11.7677
Training until validation scores don't improve for 20 rounds
[2]	valid_0's l2: 10.4593
[3]	valid_0's l2: 9.38286
[4]	valid_0's l2: 8.49365
[5]	valid_0's l2: 7.76476
[6]	valid_0's l2: 7.18413
[7]	valid_0's l2: 6.69437
[8]	valid_0's l2: 6.30222
[9]	valid_0's l2: 5.96615
[10]	valid_0's l2: 5.70453
Did not meet early stopping. Best iteration is:
[10]	valid_0's l2: 5.70453
Fold: 4
[1]	valid_0's l2: 11.9231
Training until validation scores don't improve for 20 rounds
[2]	valid_0's l2: 10.588
[3]	valid_0's l2: 9.49368
[4]	valid_0's l2: 8.60178
[5]	valid_0's l2: 7.87162
[6]	valid_0's l2: 7.28382
[7]	valid_0's l2: 6.79506
[8]	valid_0's l2: 6.40256
[9]	valid_0's l2: 6.07282
[10]	valid_0's l2: 5.80861
Did not meet early stopping. Best iteration is:
[10]	valid_0's l2: 5.80861
Fold: 5
[1]	valid_0's l2: 11.9576
Trai

TypeError: 'numpy.float64' object is not iterable

In [78]:
all_df2_test = all_df17_20[all_df17_20['date'] > '2016-04-24']

In [81]:
len(results)

28

In [90]:
all_df2_test['sales'] = test_sales

c:\users\kjb\data_analysis\10_estimate_sales_of_walmart_retail_goods\venv\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [91]:
all_df2_test.head()

,id,item_id,dept_id,cat_id,store_id,state_id,d,sales,date,wm_yr_wk,weekday,month,year,event_name_1,event_type_1,snap_CA,snap_TX,snap_WI,is_event,day,week,sell_price,lag_t1,lag_t2,lag_t3,lag_t4,lag_t5,lag_t6,lag_t7,lag_t8,lag_t9,lag_t10,lag_t11,lag_t12,lag_t13,lag_t14,lag_t15,lag_t16,lag_t17,lag_t18,lag_t19,lag_t20,lag_t21,lag_t22,lag_t23,lag_t24,lag_t25,lag_t26,lag_t27,lag_t28,rolling_mean_t7,rolling_std_t7,rolling_mean_t28,rolling_std_t28,rolling_mean_t56,rolling_std_t56,rolling_mean_t84,rolling_std_t84,rolling_mean_t112,rolling_std_t112,rolling_mean_t168,rolling_std_t168,price_change_t1,price_change_t365,rolling_price_std_t7,rolling_price_std_t28,lag_t29,lag_t30,lag_t31,lag_t32
0,14370,1437,3,1,0,0,1017,1.009044,2016-04-25,11613,1,4,2016,30,4,0,0,0,0,25,17,8.382812,1.0,1.0,0.0,3.0,1.0,1.0,1.0,0.0,3.0,1.0,1.0,0.0,0.0,0.0,2.0,1.0,0.0,3.0,2.0,4.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.142578,0.689941,1.036133,0.922363,1.017578,1.103516,0.940430,1.144531,0.910645,1.103516,0.779785,1.017578,0.0,0.0,5.960464e-08,6.216431e-02,1.0,1.0,1.0,0.0
1,14380,1438,3,1,0,0,1017,0.578026,2016-04-25,11613,1,4,2016,30,4,0,0,0,0,25,17,3.970703,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.714355,0.488037,0.357178,0.488037,0.250000,0.437012,0.226196,0.420898,0.384033,0.830078,0.380859,0.772461,0.0,0.0,0.000000e+00,0.000000e+00,1.0,1.0,1.0,1.0
2,14390,1439,3,1,0,0,1017,0.843942,2016-04-25,11613,1,4,2016,30,4,0,0,0,0,25,17,2.970703,1.0,1.0,1.0,0.0,1.0,1.0,1.0,2.0,1.0,2.0,2.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.285645,0.488037,0.535645,1.201172,0.428467,0.911621,0.321533,0.778809,0.633789,1.065430,0.643066,1.016602,0.0,0.0,0.000000e+00,0.000000e+00,0.0,1.0,1.0,0.0
3,14400,1440,3,1,0,0,1017,1.707772,2016-04-25,11613,1,4,2016,30,4,0,0,0,0,25,17,4.640625,2.0,7.0,3.0,1.0,0.0,1.0,4.0,5.0,0.0,1.0,0.0,3.0,0.0,2.0,4.0,5.0,2.0,0.0,1.0,3.0,1.0,2.0,1.0,3.0,0.0,0.0,0.0,0.0,3.000000,2.708984,1.892578,1.968750,2.000000,1.907227,2.023438,1.907227,1.892578,1.827148,1.898438,1.763672,0.0,0.0,0.000000e+00,0.000000e+00,6.0,6.0,0.0,5.0
4,14410,1441,3,1,0,0,1017,1.176830,2016-04-25,11613,1,4,2016,30,4,0,0,0,0,25,17,2.880859,4.0,2.0,2.0,2.0,1.0,1.0,0.0,1.0,1.0,2.0,1.0,1.0,0.0,2.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,4.0,1.0,0.0,4.0,4.0,0.0,1.0,0.714355,1.112305,1.107422,0.994141,1.142578,1.166992,1.011719,1.135742,1.125000,1.216797,1.136719,1.167969,0.0,0.0,0.000000e+00,5.960464e-08,0.0,0.0,0.0,0.0


In [87]:
sub = pd.read_csv('inputs/sample_submission.csv')

predictions = all_df2_test[['id', 'date', 'sales']]
predictions['id'] = list(le_id.inverse_transform(predictions['id']))


predictions = pd.pivot(predictions, index = 'id', columns = 'date', values = 'sales').reset_index()
predictions.columns = ['id'] + ['F' + str(i + 1) for i in range(28)]

evaluation_rows = [row for row in sub['id'] if 'evaluation' in row] 
evaluation = sub[sub['id'].isin(evaluation_rows)]

validation = sub[['id']].merge(predictions, on = 'id')
final = pd.concat([validation, evaluation])

for i in range(1,29):
    final['F'+str(i)] *= 1.0315
final.to_csv('submissions/submission.csv', index = False)

c:\users\kjb\data_analysis\10_estimate_sales_of_walmart_retail_goods\venv\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [92]:
final.head()

,id,F1,F2,F3,F4,F5,F6,F7,F8,F9,F10,F11,F12,F13,F14,F15,F16,F17,F18,F19,F20,F21,F22,F23,F24,F25,F26,F27,F28
0,HOBBIES_1_001_CA_1_validation,1.040829,1.008477,1.001335,0.996877,0.974703,1.052290,1.060663,0.949905,0.962422,0.985934,1.078596,1.054044,1.131717,1.164227,1.117989,0.995352,0.990711,1.006945,1.004935,1.082160,1.095978,0.991201,0.970538,0.973718,0.975546,0.987273,1.094747,1.076632
1,HOBBIES_1_002_CA_1_validation,0.596234,0.592303,0.592461,0.600610,0.590171,0.600521,0.594017,0.523600,0.514450,0.527647,0.520629,0.522556,0.517424,0.526469,0.520571,0.529376,0.529671,0.524702,0.529149,0.538248,0.536614,0.546378,0.549540,0.595673,0.599086,0.599207,0.610455,0.620122
2,HOBBIES_1_003_CA_1_validation,0.870526,0.834366,0.804685,0.817698,0.842034,0.832011,0.836123,0.772816,0.746999,0.759746,0.745706,0.799203,0.833988,0.802618,0.766931,0.747476,0.748302,0.751212,0.775410,0.789070,0.857362,0.827086,0.807831,0.799655,0.793496,0.798423,0.871123,0.845051
3,HOBBIES_1_004_CA_1_validation,1.761567,1.590045,1.579701,1.554448,1.749171,2.044780,1.935437,1.663241,1.575218,1.539044,1.607629,1.766325,2.040632,1.816862,1.708681,1.591909,1.592651,1.623299,1.731085,1.933419,1.868436,1.640311,1.531623,1.508171,1.480948,1.687078,1.932614,1.803770
4,HOBBIES_1_005_CA_1_validation,1.213901,1.090715,1.169567,1.214288,1.340411,1.456490,1.592623,1.363413,1.298521,1.210473,1.190259,1.226741,1.423685,1.379871,1.218338,1.102772,1.121450,1.114823,1.200185,1.348453,1.347771,1.105502,1.043436,1.059877,1.077720,1.162767,1.323020,1.358750


In [ ]:
len(all_df17_20)

In [51]:
item_id_gb_sell_price_mean = all_df17_20.groupby(['store_id', 'item_id', 'year', 'week'])['sell_price'].mean().rename('item_id_gb_sell_price_mean')
item_id_gb_sell_price_max = all_df17_20.groupby(['store_id', 'item_id', 'year', 'week'])['sell_price'].max().rename('item_id_gb_sell_price_max')
item_id_gb_sell_price_min = all_df17_20.groupby(['store_id', 'item_id', 'year', 'week'])['sell_price'].min().rename('item_id_gb_sell_price_min')
item_id_gb_sell_price_std = all_df17_20.groupby(['store_id', 'item_id', 'year', 'week'])['sell_price'].std().rename('item_id_gb_sell_price_std')

all_df17_20 = pd.merge(all_df17_20, item_id_gb_sell_price_mean, on=['store_id', 'item_id', 'year', 'week'], how='left')
all_df17_20 = pd.merge(all_df17_20, item_id_gb_sell_price_max, on=['store_id', 'item_id', 'year', 'week'], how='left')
all_df17_20 = pd.merge(all_df17_20, item_id_gb_sell_price_min, on=['store_id', 'item_id', 'year', 'week'], how='left')
all_df17_20 = pd.merge(all_df17_20, item_id_gb_sell_price_std, on=['store_id', 'item_id', 'year', 'week'], how='left')

item_id_gb_lag28 = all_df17_20.groupby(['store_id', 'item_id', 'year', 'week'])['lag_t28'].mean().rename('item_id_gb_lag28')
# all_df17_20 = pd.merge(all_df17_20, item_id_gb_lag28, on=['store_id', 'item_id', 'year', 'week'], how='left')

# all_df17_20['diff'] = all_df17_20['sell_price'] - all_df17_20['item_id_gb_sell_price']

In [50]:
del all_df17_20['item_id_gb_sell_price_std']

In [66]:
import pprint

In [79]:
all_df17_20.groupby(['item_id', 'month'])['sell_price'].mean()

item_id  month
0        3        2.177734
         4        2.154297
         5        2.150391
1        3        8.984375
         4        8.921875
                    ...   
3047     4        1.966797
         5        1.969727
3048     3        5.941406
         4        5.941406
         5        5.910156
Name: sell_price, Length: 9147, dtype: float16

In [76]:
# tmp = all_df17_20.groupby(['store_id', 'year','day'])['sales'].mean()

# for i in tmp.index:
#     print(i, tmp[i])

(0, 2011, 1) 1.175467366349623
(0, 2011, 2) 1.1866185634634305
(0, 2011, 3) 1.1864545752705806
(0, 2011, 4) 1.0252541816989176
(0, 2011, 5) 0.8474909806493932
(0, 2011, 6) 0.9391603804526074
(0, 2011, 7) 1.1138078058379797
(0, 2011, 8) 1.0375532961626763
(0, 2011, 9) 1.172843555264021
(0, 2011, 10) 1.177271236470974
(0, 2011, 11) 0.9255493604460479
(0, 2011, 12) 0.7853394555591997
(0, 2011, 13) 0.8568383076418498
(0, 2011, 14) 1.0288619219416202
(0, 2011, 15) 1.0969170219744178
(0, 2011, 16) 1.0090193506067564
(0, 2011, 17) 1.021318465070515
(0, 2011, 18) 0.8091177435224663
(0, 2011, 19) 0.7218760249262053
(0, 2011, 20) 0.8076418497868153
(0, 2011, 21) 0.975729747458183
(0, 2011, 22) 1.107248278123975
(0, 2011, 23) 1.3732371269268613
(0, 2011, 24) 0.9662184322728764
(0, 2011, 25) 0.7999344047228599
(0, 2011, 26) 0.6838307641849787
(0, 2011, 27) 0.6874385044276812
(0, 2011, 28) 0.7979665464086586
(0, 2011, 29) 0.7940308297802559
(0, 2011, 30) 0.9522794358806166
(0, 2011, 31) 0.844867169

(9, 2013, 10) 1.0231223351918661
(9, 2013, 11) 1.232863233847163
(9, 2013, 12) 1.155624795014759
(9, 2013, 13) 1.1608724171859626
(9, 2013, 14) 1.0665792062971466
(9, 2013, 15) 1.0360774024270252
(9, 2013, 16) 0.9020990488684815
(9, 2013, 17) 0.8366677599212856
(9, 2013, 18) 0.8878320760905215
(9, 2013, 19) 0.9385044276812069
(9, 2013, 20) 1.0534601508691375
(9, 2013, 21) 0.9773696293866841
(9, 2013, 22) 0.7195801902263037
(9, 2013, 23) 0.6539849130862578
(9, 2013, 24) 0.7582814037389308
(9, 2013, 25) 0.8960314857330273
(9, 2013, 26) 0.8558543784847491
(9, 2013, 27) 0.9567071170875697
(9, 2013, 28) 0.934240734667104
(9, 2013, 29) 0.9868809445719908
(9, 2013, 30) 1.0834699901607083
(9, 2013, 31) 1.1482453263365038
(9, 2014, 1) 0.8238766808789767
(9, 2014, 2) 1.05723187930469
(9, 2014, 3) 1.1730075434568712
(9, 2014, 4) 1.004263693014103
(9, 2014, 5) 1.0441128238766808
(9, 2014, 6) 1.0188586421777632
(9, 2014, 7) 0.8148573302722204
(9, 2014, 8) 0.8786487372909151
(9, 2014, 9) 1.031485733

In [45]:
all_df17_20.head()

,id,item_id,dept_id,cat_id,store_id,state_id,d,sales,date,wm_yr_wk,weekday,month,year,event_name_1,event_type_1,snap_CA,snap_TX,snap_WI,is_event,day,week,sell_price,revenue,lag_t28,lag_t29,lag_t30,lag_t31,lag_t32,lag_t33,lag_t34,rolling_mean_t7,rolling_std_t7,rolling_mean_t28,rolling_std_t28,rolling_mean_t56,rolling_std_t56,rolling_mean_t84,rolling_std_t84,rolling_mean_t112,rolling_std_t112,rolling_mean_t168,rolling_std_t168,price_change_t1,price_change_t365,rolling_price_std_t7,rolling_price_std_t28,revenue_lag_t28,revenue_lag_t29,revenue_lag_t30,revenue_lag_t31,revenue_lag_t32,revenue_lag_t33,revenue_lag_t34,price_lag_t1,price_lag_t2,price_lag_t3,price_lag_t4,price_lag_t5,price_lag_t6,price_lag_t28,price_lag_t29,price_lag_t30,price_lag_t31,price_lag_t32,price_lag_t33,price_lag_t34,rolling_price_std_t14,rolling_price_max_t365,price_change_t180,item_id_gb_sell_price_x,item_id_gb_sell_price_y,item_id_gb_sell_price,item_id_gb_sell_price_std
0,14370,1437,3,1,0,0,1486,0,2011-03-28,11109,1,3,2011,30,4,0,0,0,0,28,13,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,14380,1438,3,1,0,0,1486,0,2011-03-28,11109,1,3,2011,30,4,0,0,0,0,28,13,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,14390,1439,3,1,0,0,1486,0,2011-03-28,11109,1,3,2011,30,4,0,0,0,0,28,13,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,14400,1440,3,1,0,0,1486,2,2011-03-28,11109,1,3,2011,30,4,0,0,0,0,28,13,4.339844,8.679688,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.339844,4.339844,4.339844,4.339844,4.339844,4.339844,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,4.339844,4.339844,4.339844,0.0
4,14410,1441,3,1,0,0,1486,0,2011-03-28,11109,1,3,2011,30,4,0,0,0,0,28,13,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

plt.figure(figsize = (10, 7))
sns.distplot(all_df17_20['event_name_1'])

In [8]:
from sklearn.preprocessing import LabelEncoder

le_id = LabelEncoder()
le_id.fit(all_df17_20['id'])

all_df17_20['id'] = le_id.transform(all_df17_20['id'])

# 3. 피쳐 엔지니어링

## 3.1 train, test

prepare training and test data.
- 2011-01-29 ~ 2016-04-24 : d_1    ~ d_1913
- 2016-04-25 ~ 2016-05-22 : d_1914 ~ d_1941 (public)
- 2016-05-23 ~ 2016-06-19 : d_1942 ~ d_1969 (private)

# 4. 모델 제작

In [ ]:
# from sklearn.ensemble import RandomForestRegressor

# rf = RandomForestRegressor(n_estimators=100, n_jobs=-1)
# rf.fit(train2, np.log(train['target'] + 1))
# result = rf.predict(test2)

In [ ]:
# from lightgbm import LGBMRegressor
# # lgb = LGBMRegressor(num_leaves=2000, colsample_bytree=0.6, subsample=0.6, n_estimators=600, learning_rate=0.025, n_jobs=-1, device='gpu', max_bin = 63)
# lgb = LGBMRegressor(num_leaves=20, colsample_bytree=0.6, subsample=0.6, n_estimators=60, learning_rate=0.02, n_jobs=-1, device='cpu')

# lgb.fit(train, target)
# result = lgb.predict(test)

## 4.1 train set /  test set 선정

feature 선정에 있어서 다른 노트북에서는 all_df = all_df.drop(['rolling_price_max_t365', 'lag_price_t1'], inplace = True, axis = 1) 이렇게 진행했음

In [12]:
all_df17_20.columns

Index(['id', 'item_id', 'dept_id', 'cat_id', 'store_id', 'state_id', 'd',
       'sales', 'date', 'wm_yr_wk', 'weekday', 'month', 'year', 'event_name_1',
       'event_type_1', 'snap_CA', 'snap_TX', 'snap_WI', 'is_event', 'day',
       'week', 'sell_price', 'revenue', 'lag_t28', 'lag_t29', 'lag_t30',
       'lag_t31', 'lag_t32', 'lag_t33', 'lag_t34', 'rolling_mean_t7',
       'rolling_std_t7', 'rolling_mean_t28', 'rolling_std_t28',
       'rolling_mean_t56', 'rolling_std_t56', 'rolling_mean_t84',
       'rolling_std_t84', 'rolling_mean_t112', 'rolling_std_t112',
       'rolling_mean_t168', 'rolling_std_t168', 'price_change_t1',
       'price_change_t365', 'rolling_price_std_t7', 'rolling_price_std_t28',
       'revenue_lag_t28', 'revenue_lag_t29', 'revenue_lag_t30',
       'revenue_lag_t31', 'revenue_lag_t32', 'revenue_lag_t33',
       'revenue_lag_t34', 'price_lag_t1', 'price_lag_t2', 'price_lag_t3',
       'price_lag_t4', 'price_lag_t5', 'price_lag_t6', 'price_lag_t28',
       'price

In [11]:
all_df17_20.head(1)

,id,item_id,dept_id,cat_id,store_id,state_id,d,sales,date,wm_yr_wk,weekday,month,year,event_name_1,event_type_1,snap_CA,snap_TX,snap_WI,is_event,day,week,sell_price,revenue,lag_t28,lag_t29,lag_t30,lag_t31,lag_t32,lag_t33,lag_t34,rolling_mean_t7,rolling_std_t7,rolling_mean_t28,rolling_std_t28,rolling_mean_t56,rolling_std_t56,rolling_mean_t84,rolling_std_t84,rolling_mean_t112,rolling_std_t112,rolling_mean_t168,rolling_std_t168,price_change_t1,price_change_t365,rolling_price_std_t7,rolling_price_std_t28,revenue_lag_t28,revenue_lag_t29,revenue_lag_t30,revenue_lag_t31,revenue_lag_t32,revenue_lag_t33,revenue_lag_t34,price_lag_t1,price_lag_t2,price_lag_t3,price_lag_t4,price_lag_t5,price_lag_t6,price_lag_t28,price_lag_t29,price_lag_t30,price_lag_t31,price_lag_t32,price_lag_t33,price_lag_t34,rolling_price_std_t14,rolling_price_max_t365,price_change_t180,item_id_gb_sell_price,item_id_gb_lag28,diff
0,14370,1437,3,1,0,0,1486,0,2011-03-28,11109,1,3,2011,30,4,0,0,0,0,28,13,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN


In [13]:
drop_cols = ['d', 'sales', 'date', 'cat_id',  'wm_yr_wk', 'is_event' , 'event_type_1', 'lag_t32', 'lag_t33', 'lag_t34',
             'revenue','revenue_lag_t28', 'revenue_lag_t29', 'revenue_lag_t30','revenue_lag_t31', 'revenue_lag_t32', 'revenue_lag_t33', 'revenue_lag_t34',
             'price_lag_t1', 'price_lag_t2', 'price_lag_t3', 'price_lag_t4', 'price_lag_t5', 'price_lag_t6',
             'rolling_price_std_t14', 'price_lag_t28', 'price_lag_t29',
       'price_lag_t30','price_lag_t31', 'price_lag_t32', 'price_lag_t33',
       'price_lag_t34', 'price_change_t1' , 'price_change_t180',  'rolling_mean_t112' , 'rolling_std_t112'  ]

features = all_df17_20.columns.drop(drop_cols)

In [14]:
features

Index(['id', 'item_id', 'dept_id', 'store_id', 'state_id', 'weekday', 'month',
       'year', 'event_name_1', 'snap_CA', 'snap_TX', 'snap_WI', 'day', 'week',
       'sell_price', 'lag_t28', 'lag_t29', 'lag_t30', 'lag_t31',
       'rolling_mean_t7', 'rolling_std_t7', 'rolling_mean_t28',
       'rolling_std_t28', 'rolling_mean_t56', 'rolling_std_t56',
       'rolling_mean_t84', 'rolling_std_t84', 'rolling_mean_t168',
       'rolling_std_t168', 'price_change_t365', 'rolling_price_std_t7',
       'rolling_price_std_t28', 'rolling_price_max_t365',
       'item_id_gb_sell_price', 'item_id_gb_lag28', 'diff'],
      dtype='object')

In [15]:
train_set = all_df17_20[all_df17_20['date'] <= '2016-04-24']
train_set_X = train_set[features]
train_set_y = train_set['sales']

# 테스트 셋
test = all_df17_20[all_df17_20['date'] > '2016-04-24']
test_set = test[features]


var_set = all_df17_20[(all_df17_20['date'] > '2015-04-27') & (all_df17_20['date'] <= '2015-05-22')]
var_set_X = var_set[features]
var_set_y = var_set['sales']

In [ ]:
def split_store(data, store_id):
    data = data[data["store_id"] == store_id]
    return data

In [ ]:
def make_train_test_set(data, drop_cols):

    features = data.columns.drop(drop_cols)

    train_set = data[data['date'] <= '2016-04-24']
    train_set_X = train_set[features]
    train_set_y = train_set['target']

    # 테스트 셋
    test = data[data['date'] > '2016-04-24']
    test_set = test[features]

    var_set = data[(data['date'] > '2015-04-27') & (data['date'] <= '2015-05-22')]
    var_set_X = var_set[features]
    var_set_y = var_set['target']
    
    return train_set_X, train_set_y, test, test_set, var_set_X, var_set_y
    
    

In [ ]:
drop_cols = ['id', 'd', 'target', 'date', 'wm_yr_wk', 'is_event', 'lag_t28', 'lag_t29',
       'lag_t30', 'lag_t24', 'lag_t25', 'lag_t26', 'lag_t27']

train_set_X, train_set_y, test, test_set, var_set_X, var_set_y = make_train_test_set(all_df17_20, drop_cols)

In [ ]:
all_df17_20[all_df17_20.store_id == 8]

## 4.2 KFold - LGBM 모델

In [ ]:
def lgb_model(data, drop_cols):
    train_set_X, train_set_y, test, test_set, var_set_X, var_set_y = make_train_test_set(data, drop_cols)
    n_fold = 5
    folds = KFold(n_splits=n_fold, shuffle=True, random_state=333)
    splits = folds.split(train_set_X, train_set_y)

    y_preds = np.zeros(test_set.shape[0])
    y_oof = np.zeros(train_set_X.shape[0])

    feature_importances = pd.DataFrame()
    feature_importances['feature'] = train_set_X.columns
    mean_score = []


    for fold_n, (train_index, valid_index) in enumerate(splits):
        print('Fold:',fold_n+1)

        X_train, X_valid = train_set_X.iloc[train_index], train_set_X.iloc[valid_index]
        y_train, y_valid = train_set_y.iloc[train_index], train_set_y.iloc[valid_index]

        lgb = LGBMRegressor(
            boosting_type = 'gbdt',
            num_leaves = 2048,
            colsample_bytree = 0.8,
            subsample = 0.8,
            n_estimators = 300, ## 중요!!!!
            learning_rate = 0.1,
            n_jobs = -1,
            device = 'gpu',
            random_state=333
        )
        lgb.fit(X_train, y_train, eval_set=[(X_valid, y_valid)], early_stopping_rounds = 20, verbose = True)

        feature_importances[f'fold_{fold_n + 1}'] = lgb.feature_importances_

        y_pred_valid = lgb.predict(X_valid, num_iteration=lgb.best_iteration_)

        y_oof[valid_index] = y_pred_valid

        # validation 측정
        val_score = np.sqrt(metrics.mean_squared_error(y_pred_valid, y_valid))
        print(f'val rmse score is {val_score}')
        mean_score.append(val_score)

        all_pred = lgb.predict(var_set_X, num_iteration=lgb.best_iteration_)
        all_var_score = np.sqrt(metrics.mean_squared_error(all_pred, var_set_y))
        print(f'2015-04-27 부터 2015-05-22까지 데이터로 validation rmse 결과: {all_var_score}')


        # 예측
        y_preds += lgb.predict(test_set, num_iteration=lgb.best_iteration_) / n_fold

        # 메모리 정리
        del X_train, X_valid, y_train, y_valid

    features = train_set_X.columns
    params = lgb.get_params()
    eval_results = lgb.evals_result_['valid_0']['l2']

    print('mean rmse score over folds is',np.mean(mean_score))
    test['sales'] = y_preds
    write_params_features(features, params, eval_results)
    save_feature_importance(feature_importances)
    
    return test

In [17]:
n_fold = 5
folds = KFold(n_splits=n_fold, shuffle=True)
splits = folds.split(train_set_X, train_set_y)

y_preds = np.zeros(test_set.shape[0])
y_oof = np.zeros(train_set_X.shape[0])

feature_importances = pd.DataFrame()
feature_importances['feature'] = train_set_X.columns
mean_score = []

print(train_set_X.columns)

for fold_n, (train_index, valid_index) in enumerate(splits):
    print('Fold:',fold_n+1)
    
    X_train, X_valid = train_set_X.iloc[train_index], train_set_X.iloc[valid_index]
    y_train, y_valid = train_set_y.iloc[train_index], train_set_y.iloc[valid_index]
    
    lgb = LGBMRegressor(
        objective = 'regression',
        boosting_type = 'gbdt',
        num_leaves = 4096,
        colsample_bytree = 0.8,
        subsample = 0.8,
        n_estimators = 400, ## 중요!!!!
        learning_rate = 0.1,
        n_jobs = -1,
        device = 'gpu',
        reg_lambda = 0.1
    )
    lgb.fit(X_train, y_train, eval_set=[(X_valid, y_valid)], early_stopping_rounds = 20, verbose = True)

    feature_importances[f'fold_{fold_n + 1}'] = lgb.feature_importances_
    
    y_pred_valid = lgb.predict(X_valid, num_iteration=lgb.best_iteration_)
    
    y_oof[valid_index] = y_pred_valid
    
    # validation 측정
    val_score = np.sqrt(metrics.mean_squared_error(y_pred_valid, y_valid))
    print(f'val rmse score is {val_score}')
    mean_score.append(val_score)
    
    all_pred = lgb.predict(var_set_X, num_iteration=lgb.best_iteration_)
    all_var_score = np.sqrt(metrics.mean_squared_error(all_pred, var_set_y))
    print(f'2015-04-27 부터 2015-05-22까지 데이터로 validation rmse 결과: {all_var_score}')
    
    
    # 예측
    y_preds += lgb.predict(test_set, num_iteration=lgb.best_iteration_) / n_fold
    
    # 메모리 정리
    del X_train, X_valid, y_train, y_valid

    
params = lgb.get_params()
eval_results = lgb.evals_result_['valid_0']['l2']

print('mean rmse score over folds is',np.mean(mean_score))
test['sales'] = y_preds



Index(['id', 'item_id', 'dept_id', 'store_id', 'state_id', 'weekday', 'month',
       'year', 'event_name_1', 'snap_CA', 'snap_TX', 'snap_WI', 'day', 'week',
       'sell_price', 'lag_t28', 'lag_t29', 'lag_t30', 'lag_t31',
       'rolling_mean_t7', 'rolling_std_t7', 'rolling_mean_t28',
       'rolling_std_t28', 'rolling_mean_t56', 'rolling_std_t56',
       'rolling_mean_t84', 'rolling_std_t84', 'rolling_mean_t168',
       'rolling_std_t168', 'price_change_t365', 'rolling_price_std_t7',
       'rolling_price_std_t28', 'rolling_price_max_t365',
       'item_id_gb_sell_price', 'item_id_gb_lag28', 'diff'],
      dtype='object')
Fold: 1
[1]	valid_0's l2: 11.4508
Training until validation scores don't improve for 20 rounds
[2]	valid_0's l2: 10.0644
[3]	valid_0's l2: 8.93787
[4]	valid_0's l2: 8.08352
[5]	valid_0's l2: 7.32279
[6]	valid_0's l2: 6.76024
[7]	valid_0's l2: 6.24948
[8]	valid_0's l2: 5.83694
[9]	valid_0's l2: 5.49328
[10]	valid_0's l2: 5.21453
[11]	valid_0's l2: 5.00862
[12]	valid_

[127]	valid_0's l2: 4.04351
[128]	valid_0's l2: 4.04384
[129]	valid_0's l2: 4.04436
[130]	valid_0's l2: 4.04328
[131]	valid_0's l2: 4.04065
[132]	valid_0's l2: 4.04061
[133]	valid_0's l2: 4.04033
[134]	valid_0's l2: 4.0406
[135]	valid_0's l2: 4.04042
[136]	valid_0's l2: 4.03997
[137]	valid_0's l2: 4.04009
[138]	valid_0's l2: 4.03992
[139]	valid_0's l2: 4.03974
[140]	valid_0's l2: 4.04029
[141]	valid_0's l2: 4.0398
[142]	valid_0's l2: 4.0398
[143]	valid_0's l2: 4.03945
[144]	valid_0's l2: 4.03916
[145]	valid_0's l2: 4.03882
[146]	valid_0's l2: 4.03891
[147]	valid_0's l2: 4.03892
[148]	valid_0's l2: 4.03909
[149]	valid_0's l2: 4.03912
[150]	valid_0's l2: 4.03886
[151]	valid_0's l2: 4.03856
[152]	valid_0's l2: 4.03855
[153]	valid_0's l2: 4.03846
[154]	valid_0's l2: 4.03862
[155]	valid_0's l2: 4.03818
[156]	valid_0's l2: 4.03767
[157]	valid_0's l2: 4.03756
[158]	valid_0's l2: 4.03771
[159]	valid_0's l2: 4.03716
[160]	valid_0's l2: 4.03752
[161]	valid_0's l2: 4.03754
[162]	valid_0's l2: 4.0

[32]	valid_0's l2: 4.02625
[33]	valid_0's l2: 4.0177
[34]	valid_0's l2: 4.00847
[35]	valid_0's l2: 3.99913
[36]	valid_0's l2: 3.98916
[37]	valid_0's l2: 3.98214
[38]	valid_0's l2: 3.97414
[39]	valid_0's l2: 3.96769
[40]	valid_0's l2: 3.96127
[41]	valid_0's l2: 3.95609
[42]	valid_0's l2: 3.95163
[43]	valid_0's l2: 3.94907
[44]	valid_0's l2: 3.94581
[45]	valid_0's l2: 3.94131
[46]	valid_0's l2: 3.93698
[47]	valid_0's l2: 3.92972
[48]	valid_0's l2: 3.92814
[49]	valid_0's l2: 3.92209
[50]	valid_0's l2: 3.91839
[51]	valid_0's l2: 3.91501
[52]	valid_0's l2: 3.91434
[53]	valid_0's l2: 3.90762
[54]	valid_0's l2: 3.90228
[55]	valid_0's l2: 3.90042
[56]	valid_0's l2: 3.89898
[57]	valid_0's l2: 3.89711
[58]	valid_0's l2: 3.89422
[59]	valid_0's l2: 3.89207
[60]	valid_0's l2: 3.88604
[61]	valid_0's l2: 3.88293
[62]	valid_0's l2: 3.88108
[63]	valid_0's l2: 3.87845
[64]	valid_0's l2: 3.87758
[65]	valid_0's l2: 3.8771
[66]	valid_0's l2: 3.87482
[67]	valid_0's l2: 3.8741
[68]	valid_0's l2: 3.87274
[69]

c:\users\kjb\data_analysis\10_estimate_sales_of_walmart_retail_goods\venv\lib\site-packages\ipykernel_launcher.py:61: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [18]:
feature_importances

,feature,fold_1,fold_2,fold_3,fold_4,fold_5
0,id,16812,26838,21421,15272,17485
1,item_id,17834,24656,20783,15806,17837
2,dept_id,741,1071,890,696,757
3,store_id,13331,20471,16574,11860,13157
4,state_id,2657,3990,3287,2408,2582
5,weekday,12465,19496,16093,10702,12312
6,month,2744,3445,3029,2449,2751
7,year,7803,11455,9349,7100,7665
8,event_name_1,1502,2391,1931,1344,1569
9,snap_CA,1259,1769,1617,1297,1324


In [19]:
mean_score

[1.9116298195516142,
 2.008743902930001,
 1.9249074828307364,
 1.9556963196441517,
 1.9458363652957718]

## 4.5 베이지안 최적화(Bayesian optimization
:  초매개변수(Hyper parameter)를 자동으로 조정

In [29]:
from bayes_opt import BayesianOptimization
from functools import partial   

In [34]:
def lgb_cv(num_leaves, learning_rate, n_estimators, subsample, colsample_bytree, reg_alpha, reg_lambda, x_data=None, y_data=None, n_splits=5, output='score'):
    score = 0
    kf = KFold(n_splits=n_splits)
    models = []
    for train_index, valid_index in kf.split(x_data):
        x_train, y_train = x_data.iloc[train_index], y_data[train_index]
        x_valid, y_valid = x_data.iloc[valid_index], y_data[valid_index]
        
        model = LGBMRegressor(
            num_leaves = int(num_leaves), 
            learning_rate = learning_rate, 
            n_estimators = int(n_estimators), 
            subsample = np.clip(subsample, 0, 1), 
            colsample_bytree = np.clip(colsample_bytree, 0, 1), 
            reg_alpha = reg_alpha, 
            reg_lambda = reg_lambda,
            device = 'gpu'
        )
        
        model.fit(x_train, y_train)
        models.append(model)
        
        pred = model.predict(x_valid)
        true = y_valid
        score += metrics.mean_squared_error(true, pred)/n_splits
    
    if output == 'score':
        return score
    if output == 'model':
        return models

In [35]:
# 모델과 관련없는 변수 고정
func_fixed = partial(lgb_cv, x_data=train_set_X.reset_index(drop=True), y_data=train_set_y.reset_index(drop=True), n_splits=5, output='score') 
# 베이지안 최적화 범위 설정
lgbBO = BayesianOptimization(
    func_fixed, 
    {
        'num_leaves': (64, 4096),        # num_leaves,       범위(16~1024)
        'learning_rate': (0.0001, 0.1),  # learning_rate,    범위(0.0001~0.1)
        'n_estimators': (16, 1024),      # n_estimators,     범위(16~1024)
        'subsample': (0.4, 1),             # subsample,        범위(0~1)
        'colsample_bytree': (0.4, 1),      # colsample_bytree, 범위(0~1)
        'reg_alpha': (0, 10),            # reg_alpha,        범위(0~10)
        'reg_lambda': (0, 50),           # reg_lambda,       범위(0~50)
    }, 
    random_state=4321                    # 시드 고정
)
lgbBO.maximize(init_points=5, n_iter=30) # 처음 5회 랜덤 값으로 score 계산 후 30회 최적화

# 이 예제에서는 7개 하이퍼 파라미터에 대해 30회 조정을 시도했습니다.
# 다양한 하이퍼 파라미터, 더 많은 iteration을 시도하여 최상의 모델을 얻어보세요!

|   iter    |  target   | colsam... | learni... | n_esti... | num_le... | reg_alpha | reg_la... | subsample |
-------------------------------------------------------------------------------------------------------------


KeyboardInterrupt: 

In [ ]:
feature_importances

## 4.3 store 별로 모델 만들기

In [ ]:
drop_cols = ['id', 'd', 'target', 'date', 'wm_yr_wk', 'is_event', 'lag_t28', 'lag_t29',
       'lag_t30', 'lag_t24', 'lag_t25', 'lag_t26', 'lag_t27']

test_sets = []
for i in range(10):
    store_i = split_store(all_df17_20, i)
    test_sets.append(lgb_model(store_i, drop_cols))

In [ ]:
for i in range(1, 10):
    test_sets[0] = pd.concat([test_sets[0], test_sets[i]])

In [ ]:
test = test_sets[0].sort_index()

## 4.4 keras 모델

In [ ]:
from keras import backend as K

def root_mean_squared_error(y_true, y_pred):
        return K.sqrt(K.mean(K.square(y_pred - y_true))) 

# model.compile(optimizer = "rmsprop", loss = root_mean_squared_error, 
#               metrics =["accuracy"])

In [ ]:
from keras.models import Sequential             # Sequential 생성자를 불러옵니다.
from keras.layers import Dense, Activation, Dropout      # Dense와 Activation 두 층 인스턴스를 불러옵니다.

model = Sequential()
model.add(Dense(128, activation='relu', kernel_initializer='he_normal', input_dim=train_set_X[features].shape[1]))
model.add(Dropout(0.2))
model.add(Dense(128, activation='relu', kernel_initializer='he_normal'))
model.add(Dropout(0.2))
model.add(Dense(128, activation='relu', kernel_initializer='he_normal'))
# model.add(Dense(4, activation='relu'))
model.add(Dense(1, activation='linear'))                   # 1차원 출력에 'sigmoid' 함수를 적용하는 Dense 층입니다.
model.compile(optimizer='rmsprop',                          # 최적화 함수 = 'rmsprop'
              loss= 'mse',                   # 손실 함수 = 'binary_crossentropy'
              metrics=[root_mean_squared_error])                         # 평가 지표 = 'accuracy'

# 학습시키기 
model.fit(train_set_X, train_set_y, epochs=1, batch_size=1000, validation_split = 0.2)           # 생성된 데이터를 32개씩의 배치로 나누어 전체를 총 10회 학습시킵니다.

In [ ]:
test_set.head()

In [ ]:
result_ = model.predict(test_set)

In [ ]:
result_

In [ ]:
test['target'] = result_

# 5. 예측 및 제출

In [20]:
sub = pd.read_csv('inputs/sample_submission.csv')

predictions = test[['id', 'date', 'sales']]
predictions['id'] = list(le_id.inverse_transform(predictions['id']))


predictions = pd.pivot(predictions, index = 'id', columns = 'date', values = 'sales').reset_index()
predictions.columns = ['id'] + ['F' + str(i + 1) for i in range(28)]

evaluation_rows = [row for row in sub['id'] if 'evaluation' in row] 
evaluation = sub[sub['id'].isin(evaluation_rows)]

validation = sub[['id']].merge(predictions, on = 'id')
final = pd.concat([validation, evaluation])

for i in range(1,29):
    final['F'+str(i)] *= 1.0315
final.to_csv('submissions/submission.csv', index = False)

c:\users\kjb\data_analysis\10_estimate_sales_of_walmart_retail_goods\venv\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [21]:
len(final)

60980

In [22]:
final.head()

,id,F1,F2,F3,F4,F5,F6,F7,F8,F9,F10,F11,F12,F13,F14,F15,F16,F17,F18,F19,F20,F21,F22,F23,F24,F25,F26,F27,F28
0,HOBBIES_1_001_CA_1_validation,0.873323,0.784295,0.779143,0.783360,0.950387,1.206925,1.239522,0.890044,0.899469,0.877505,0.760785,0.928704,1.158570,0.947115,0.893189,0.763532,0.765772,0.770928,0.904047,1.172698,1.197379,0.857586,0.767958,0.770748,0.806577,0.908495,1.195651,1.230713
1,HOBBIES_1_002_CA_1_validation,0.194555,0.176493,0.175126,0.187375,0.222587,0.294166,0.329015,0.227524,0.216930,0.211489,0.205290,0.248227,0.316785,0.263612,0.223927,0.209645,0.208626,0.205935,0.240654,0.316128,0.318263,0.227395,0.212678,0.239405,0.236594,0.273997,0.354547,0.359255
2,HOBBIES_1_003_CA_1_validation,0.543399,0.497084,0.495997,0.498013,0.580580,0.807756,0.808906,0.607966,0.669579,0.659603,0.564589,0.665952,0.882347,0.757665,0.668947,0.630876,0.633323,0.583154,0.663205,0.820275,0.862737,0.640946,0.580865,0.571355,0.532724,0.587334,0.766823,0.768947
3,HOBBIES_1_004_CA_1_validation,1.685035,1.456727,1.451771,1.479555,1.855729,2.302845,2.426384,1.847484,1.737332,1.600452,1.532305,1.922024,2.372421,1.831755,1.881976,1.570869,1.589190,1.551792,1.744828,2.172816,2.424101,1.851453,1.598248,1.531185,1.484638,1.844668,2.472557,2.483112
4,HOBBIES_1_005_CA_1_validation,1.123947,1.039583,1.056198,1.021045,1.158401,1.514998,1.630911,1.143672,1.075495,0.974368,0.972741,1.171539,1.505801,1.184925,1.162892,1.019490,1.048211,1.009605,1.197927,1.496557,1.526300,1.071112,0.961041,0.946657,0.970736,1.171604,1.524786,1.609267


In [ ]:
# 새롭게 구한 값도 보정하니 정확도가 높아짐
# 1.04 를 곱하라고 했는데 1.03 곱하니 더 정확도가 높아짐
# 이차회귀식으로 계산 결과, 1.0315 곱하는게 가장 좋음.
# tmp = pd.read_csv('submissions/sub_dt_lgb.csv')
# for i in range(1,29):
#     tmp['F'+str(i)] *= 1.0315
# tmp.to_csv('submissions/submission.csv', index = False)

https://www.kaggle.com/c/m5-forecasting-accuracy/submit

In [23]:
time.sleep(2)
os.chdir("submissions")
!kaggle competitions submit -c m5-forecasting-accuracy -f submission.csv -m lgb
os.chdir("../")

Successfully submitted to M5 Forecasting - Accuracy



  0%|          | 0.00/20.6M [00:00<?, ?B/s]
  0%|          | 8.00k/20.6M [00:00<09:59, 35.9kB/s]
  0%|          | 96.0k/20.6M [00:00<07:06, 50.3kB/s]
  2%|1         | 360k/20.6M [00:00<04:57, 71.2kB/s] 
  7%|6         | 1.42M/20.6M [00:00<03:18, 101kB/s]
 11%|#         | 2.21M/20.6M [00:00<02:13, 144kB/s]
 13%|#2        | 2.62M/20.6M [00:03<02:07, 148kB/s]
 31%|###1      | 6.48M/20.6M [00:03<01:09, 211kB/s]
 38%|###8      | 7.84M/20.6M [00:03<00:44, 299kB/s]
 44%|####4     | 9.12M/20.6M [00:03<00:28, 423kB/s]
 50%|#####     | 10.4M/20.6M [00:03<00:18, 595kB/s]
 56%|#####6    | 11.5M/20.6M [00:03<00:11, 832kB/s]
 62%|######1   | 12.7M/20.6M [00:04<00:07, 1.15MB/s]
 67%|######7   | 13.9M/20.6M [00:04<00:04, 1.58MB/s]
 73%|#######3  | 15.0M/20.6M [00:04<00:02, 2.14MB/s]
 79%|#######8  | 16.2M/20.6M [00:04<00:01, 2.83MB/s]
 84%|########4 | 17.3M/20.6M [00:04<00:00, 3.67MB/s]
 90%|########9 | 18.4M/20.6M [00:04<00:00, 4.62MB/s]
 95%|#########5| 19.6M/20.6M [00:04<00:00, 5.65MB/s]
100%|####

# 6. 모델 파라미터 및 피처 기록 및 모델 저장하기

In [24]:
write_params_features(features, params, eval_results, mean_score)

In [25]:
save_feature_importance(feature_importances)

C:\Users\KJB\data_analysis\10_estimate_sales_of_walmart_retail_goods\mypackage\utils.py:134: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  fi = pd.concat([fi, tmp])
C:\Users\KJB\data_analysis\10_estimate_sales_of_walmart_retail_goods\mypackage\utils.py:135: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  fi = pd.concat([fi, feature])


In [ ]:
## 1등 노트북이랑 비교해서 카테고리형 피처 빠진거있나 확인하기